In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from datetime import date, datetime
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

# 1. DataFrame 생성

### SparkSession.createDataFrame(data, schema=None, samplingRatio=None, verifySchema=True)
- data : RDD or iterable
- **schema : pyspark.sql.types.DataType, str or list, optional**
    - 지정하지 않으면 기본으로 생성해줌
- **samplingRatio : the sample ratio of rows used for inferring**
    - 스키마가 입력되지 않으면 데이터로 유추해야함, 이때 확인할 데이터 비율
        - 인수값이 None이면 전달된 data의 첫번째 data만 읽어 스키마를 유추하게 됨.
        - 단, 첫번째 data가 컬렴명이면 유추가 불가능해져서 에러 발생
- verifySchema : verify data types of every row against schema. Enabled by default

- SparkSession 객체를 사용해 DataFrame을 생성할 수 있다.
- SparkSession 객체는 pyspark shell을 실행할 때 spark 라는 이름으로 미리 생성된다.

## Row 객체를 사용해 생성하기

- row : DataFrame에서의 한 행

In [3]:
# !pip install pandas
# !pip install pyarrow

In [4]:
import pandas as pd
from datetime import date, datetime
from pyspark.sql import *

In [5]:
##Spark.Row 클래스
??Row

Init signature: Row(*args: Optional[str], **kwargs: Optional[Any]) -> 'Row'
Source:        
class Row(tuple):

    """
    A row in :class:`DataFrame`.
    The fields in it can be accessed:

    * like attributes (``row.key``)
    * like dictionary values (``row[key]``)

    ``key in row`` will search through row keys.

    Row can be used to create a row object by using named arguments.
    It is not allowed to omit a named argument to represent that the value is
    None or missing. This should be explicitly set to None in this case.

    .. versionchanged:: 3.0.0
        Rows created from named arguments no longer have
        field names sorted alphabetically and will be ordered in the position as
        entered.

    Examples
    --------
    >>> from pyspark.sql import Row
    >>> row = Row(name="Alice", age=11)
    >>> row
    Row(name='Alice', age=11)
    >>> row['name'], row['age']
    ('Alice', 11)
    >>> row.name, row.age
    ('Alice', 11)
    >>> 'name' in row
    True
  

In [8]:
# spark.Row는 명명된 인수를 사용하여 행 개체를 만드는 데 사용할 수 있음
row = Row(name="김철수", age=15, birth=date(2011, 7, 22))
row
row["name"]

Row(name='김철수', age=15, birth=datetime.date(2011, 7, 22))

'김철수'

In [12]:
# spark DF 생성
# row class 의 생성자로 keyword args를 선달해서 생성
df = spark.createDataFrame([
    Row(name="김철수", age=15, birth=date(2011, 7, 22)),
    Row(name="이제동", age=17, birth=date(2009, 1, 1)),
    Row(name="김명운", age=19, birth=date(2007, 11, 30))
])
df
# 출력 결과 : DataFrame[name: string, age: bigint, birth: date] 
# --> 지연연산되면서 스키마(기본 구조 : 컬럼명:데이터타입)

df.show()
# 출력결과
# +------+---+----------+
# |  name|age|     birth|
# +------+---+----------+
# |김철수| 15|2011-07-22|
# |이제동| 17|2009-01-01|
# |김명운| 19|2007-11-30|
# +------+---+----------+
# df 확인 : action을 진행해야함 --> show(n) : n개의 행만큼만 출력, n 기본값 20개

DataFrame[name: string, age: bigint, birth: date]

+------+---+----------+
|  name|age|     birth|
+------+---+----------+
|김철수| 15|2011-07-22|
|이제동| 17|2009-01-01|
|김명운| 19|2007-11-30|
+------+---+----------+



- 논리적 연산 계획을 최적화하기 위해 schema 사용
- 지정하지 않으면 자동생성
- 전체 스키마 확인
    - df.printSchema()

In [16]:
# 스키마(구조) 확인 - 자동 생성된(data를 보고 spark가 유추해놓은)
# + null 허용 여부까지 // 대부분 허용하게 됨
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- birth: date (nullable = true)



## schema를 명시하여 DataFrame 생성
- 사용자 명시 스키마 활용
- schema 인수 사용

In [21]:
# 튜플에 데이터를 저장하고 스키마(pands df의 column)를 직접 지정
df2 = spark.createDataFrame([
    Row(name="김철수", age=15, birth=date(2011, 7, 22)),
    Row(name="이제동", age=17, birth=date(2009, 1, 1)),
    Row(name="김명운", age=19, birth=date(2007, 11, 30))
], schema='name string, age int, birth date')
df2.show()
df2.printSchema()

+------+---+----------+
|  name|age|     birth|
+------+---+----------+
|김철수| 15|2011-07-22|
|이제동| 17|2009-01-01|
|김명운| 19|2007-11-30|
+------+---+----------+

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- birth: date (nullable = true)



## StructType 객체를 사용해 Schema 지정

In [22]:
??StructField

Init signature:
StructField(
    name: str,
    dataType: pyspark.sql.types.DataType,
    nullable: bool = True,
    metadata: Optional[Dict[str, Any]] = None,
)
Source:        
class StructField(DataType):
    """A field in :class:`StructType`.

    Parameters
    ----------
    name : str
        name of the field.
    dataType : :class:`DataType`
        :class:`DataType` of the field.
    nullable : bool, optional
        whether the field can be null (None) or not.
    metadata : dict, optional
        a dict from string to simple type that can be toInternald to JSON automatically

    Examples
    --------
    >>> from pyspark.sql.types import StringType, StructField
    >>> (StructField("f1", StringType(), True)
    ...      == StructField("f1", StringType(), True))
    True
    >>> (StructField("f1", StringType(), True)
    ...      == StructField("f2", StringType(), True))
    False
    """

    def __init__(
        self,
        name: str,
        dataType: DataType,
       

In [23]:
from pyspark.sql.types import *

In [40]:
# StructField(name: ,dataType: ,nullable: bool = True, metadata: None)
# df의 열별로 structFiled를 구성
# 컬럼 type은 XXXType() 사용 : StringType(), IntegerType()

schema = StructType([
    StructField("name", StringType(), False),
    StructField("age", IntegerType(), False),
    StructField("birth", DateType(), False),
])
data = [
    ("김철수", 15, date(2011, 7, 22)),
    # ("", 15, date(2011, 7, 22)),
    # 빈문자열은 null과 다르기때문에 error없이 수행 가능
    ("이제동", 17, date(2009, 1, 1)),
    # ("이제동", "", date(2009, 1, 1)),
    # PySparkTypeError: [CANNOT_ACCEPT_OBJECT_IN_TYPE] `IntegerType()` can not accept object `` in type `str`.
    ("김명운", 19, date(2007, 11, 30)),
    # ("김명운", 19, None),
    # PySparkValueError: [CANNOT_BE_NONE] Argument `obj` can not be None.
]

In [41]:
df3 = spark.createDataFrame(data=data, schema=schema)
df3.printSchema()
df3.show()

root
 |-- name: string (nullable = false)
 |-- age: integer (nullable = false)
 |-- birth: date (nullable = false)

+------+---+----------+
|  name|age|     birth|
+------+---+----------+
|김철수| 15|2011-07-22|
|이제동| 17|2009-01-01|
|김명운| 19|2007-11-30|
+------+---+----------+



## 중첩스키마적용
- 컬럼의 데이터가 단일 데이터가 아닌 iter 데이터일 경우
- -스키마를 컬럼의 원소값 각각에 대해 생성 가능

In [43]:
data = [
    ('김철수', 15, date(2022,7,22), ('010','1111','2222')),
    ('이제동', 20, date(2021,7,22), ('010','2222','3333')),
    ('김명운', 25, date(2020,7,22), ('010','4444','5555'))
]
# StructField(name, dataType, nullable=True, metadata=None)

schema = StructType([
    StructField('name',StringType(), False, {"desc":"이름"}), # name 컬럼 null 불허
    StructField('age',IntegerType(), False, {"desc":"나이"}),
    StructField('birth',DateType(), False, {"desc":"생일"}),
    StructField('phone',StructType([
        StructField("phone1", StringType(), True),
        StructField("phone2", StringType(), True),
        StructField("phone3", StringType(), True)
    ]), False, {"desc":"전화번호"}),
])

In [44]:
df4 = spark.createDataFrame(data=data, schema=schema)

In [45]:
df4.printSchema()
df4.show()

root
 |-- name: string (nullable = false)
 |-- age: integer (nullable = false)
 |-- birth: date (nullable = false)
 |-- phone: struct (nullable = false)
 |    |-- phone1: string (nullable = true)
 |    |-- phone2: string (nullable = true)
 |    |-- phone3: string (nullable = true)

+------+---+----------+-----------------+
|  name|age|     birth|            phone|
+------+---+----------+-----------------+
|김철수| 15|2022-07-22|{010, 1111, 2222}|
|이제동| 20|2021-07-22|{010, 2222, 3333}|
|김명운| 25|2020-07-22|{010, 4444, 5555}|
+------+---+----------+-----------------+



#### schema 접근
    - df.schema
    - 주로 json() 이용해서 json으로 변환 후, 저장하는 방식을 사용함

In [48]:
# 스키마를 json으로 변환하여 확인
df4.schema
print('-----------------------------------------------------')
# 이름 => \uc774\ub984  유니코드로 나온다. 한글인식문제
df4_json = df4.schema.json()
print(df4_json)
print('-----------------------------------------------------')
# encode() 로 바이트문자(코드)로 변환한 후,
# decode() 로 다시 디코딩할때 바로 볼 수 있도록 unicode_escape 옵션 추가
print(df4_json.encode().decode('unicode_escape'))

StructType([StructField('name', StringType(), False), StructField('age', IntegerType(), False), StructField('birth', DateType(), False), StructField('phone', StructType([StructField('phone1', StringType(), True), StructField('phone2', StringType(), True), StructField('phone3', StringType(), True)]), False)])

-----------------------------------------------------
{"fields":[{"metadata":{"desc":"\uc774\ub984"},"name":"name","nullable":false,"type":"string"},{"metadata":{"desc":"\ub098\uc774"},"name":"age","nullable":false,"type":"integer"},{"metadata":{"desc":"\uc0dd\uc77c"},"name":"birth","nullable":false,"type":"date"},{"metadata":{"desc":"\uc804\ud654\ubc88\ud638"},"name":"phone","nullable":false,"type":{"fields":[{"metadata":{},"name":"phone1","nullable":true,"type":"string"},{"metadata":{},"name":"phone2","nullable":true,"type":"string"},{"metadata":{},"name":"phone3","nullable":true,"type":"string"}],"type":"struct"}}],"type":"struct"}
-----------------------------------------------------
{"fields":[{"metadata":{"desc":"이름"},"name":"name","nullable":false,"type":"string"},{"metadata":{"desc":"나이"},"name":"age","nullable":false,"type":"integer"},{"metadata":{"desc":"생일"},"name":"birth","nullable":false,"type":"date"},{"metadata":{"desc":"전화번호"},"name":"phone","nullable":false,"type":{"fi

In [55]:
#- spark.df의 각 컬럼 data type 확인의 경우
#   - pandas.DataFrame과 같이 df.dtypes 로 확인 
df4.dtypes

[('name', 'string'),
 ('age', 'int'),
 ('birth', 'date'),
 ('phone', 'struct<phone1:string,phone2:string,phone3:string>')]

## Pandas DataFrame으로 생성
- pd.DataFrame을 spark DataFrame으로 변환
- spark.createDataFrame(pd.DF) 이렇게 넘겨주는 방식
    - 데이터 프레임 자체를 그냥 넘기는게 아니라
    - pd.DataFrame.**iteritems** 속성값을 전달해야 함
        - pd.iteritems는 pd.DataFrame의 items라는 속성에 값이 들어 있음
            - 현재 판다스 버전에서 변경사항이 있어서 구글링해서 찾으면 과거 코드만 나와버리는 중
        - 그래서 pd.DataFrame.iteritems = pd.DataFrame.items 이렇게 불러다가 저장해줘야함

In [58]:
pandas_df = pd.DataFrame({
    'name':['김철수','이제동','김명운'],
    'age':[20, 21, 22],
    'birth':[date(2022,7,1),date(2022,7,2),date(2022,7,3)]
})

type(pandas_df)
pandas_df

pandas.core.frame.DataFrame

,name,age,birth
0,김철수,20,2022-07-01
1,이제동,21,2022-07-02
2,김명운,22,2022-07-03


In [59]:
## pandas 2.0 버전 이상부터 iteritems atrr이 items로 변경됨
# sprk.createDataFrame은 pd.DataFrame.iteritems를 사용하므로 변경 반영 후 사용해야 함
# pandas에 저장되어 있는 속성값을 직접 설정하는 코드
pd.DataFrame.iteritems = pd.DataFrame.items

In [60]:
df_pd_sp = spark.createDataFrame(pandas_df)
df_pd_sp
df_pd_sp.show()

DataFrame[name: string, age: bigint, birth: date]

+------+---+----------+
|  name|age|     birth|
+------+---+----------+
|김철수| 20|2022-07-01|
|이제동| 21|2022-07-02|
|김명운| 22|2022-07-03|
+------+---+----------+



## spark.DataFrame -> Pandas.DataFrame
- 스파크의 DataFrame을 사용하는 것이 성능상 더 이득
- 스파크는 병렬처리도 해주고... 쿼리실행 최적화도 해주고...
- =============================================================
- 하지만 스파크 api가 Pandas에 비해 제공되는 기능(module)이 적어서
- Pandas를 써야만 해결이 가능하다면 Pandas로 가공 이후 스파크 DataFrame으로 변환도 가능

## spark.DataFrame -> pandas.DataFrame
- sparkDF.toPandas()

In [62]:
df_pd_sp= df_pd_sp.toPandas()
df_pd_sp

,name,age,birth
0,김철수,20,2022-07-01
1,이제동,21,2022-07-02
2,김명운,22,2022-07-03


## pandas.DataFrame -> spark.DataFrame
- pandasDF.to_pandas_on_spark()
    - pyarrow timezone을 무시하도록 변경
    - numpy 2.0 이상 버전에서는 에러 발생
    - 쓰려면 numpy 2.0 미만으로 downgrade 해야 함

In [75]:
# ! pip list

In [72]:
# !pip uninstall numpy -y
# !pip install "numpy<2.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 1.7 MB/s eta 0:00:00m eta 0:00:010:00:01


In [76]:
import os
os.environ['PYARROW_IGNORE_TIMEZONE']='1'

In [78]:
# sparkDF -> pysparkDF -> pandasDF 이렇게 구성이 있는데
# 대체로 sparkDF -> pandasDF 이런식으로 바꿔서 데이터 처리하는데
# sparkDF -> pysparkDF 이 상태에서 데이터를 처리하는 것이 더 좋을때가 있어서 
# 지금 numpy 2.0미만으로 다운그레이드해서 아래의 모듈들을 써보려는 과정임

# 모듈이 오래됨, pyspark df를 활용가능한 모듈이기때문에 파이프라인 ETL위래서 모듈 사용가능한 상황으로 구성
df_pd_sp.to_pandas_on_spark()

AttributeError: 'DataFrame' object has no attribute 'to_pandas_on_spark'

## 외부파일을 사용해 DataFrame 생성

In [5]:
class_df = spark.read.csv('/dataframe/a_class_info.csv', header = True)
type(class_df)
class_df.show(3)

pyspark.sql.dataframe.DataFrame

+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     6OL| ANKYI|           20|   Urban| Non-public|     Standard|
|     ZNS| ANKYI|           21|   Urban| Non-public|     Standard|
|     2B1| CCAAW|           18|Suburban| Non-public| Experimental|
+--------+------+-------------+--------+-----------+-------------+
only showing top 3 rows



### pyspark.sql.dataframe.DataFrame show()
- def show( : 20개의 행을 표시)
- def show(numRows : scala.Int) : 정해진 수 만큼 행 표시
- def show(truncate : scala.Boolean) : 열값이 길어 모두 표현되지 않을경우 표현 여부
    - truncate : True -> 열값을 자르고 표시 / False -> 열값을 모두 표시
- def show(numRows : scala.Int, truncate : scala.Boolean) : 표현할 행과 열값을 자를것인지의 여부
- def show(numRows : scala.Int, truncate : scala.Int ) : 표현할 행과 열값을 몇 글자 보여줄 것인지 여
- def show(numRows : scala.Int, truncate : scala.Int, vertical : scala.Boolean) : 레코드별로 세로로 표시할 
- 모든 행을 표현하고자 한다면
    - count()사용해 행 수를 얻어와서 show()로 연결해야 함
    - def show(df.count())것인지의 여부

In [18]:
# class_df.show()
class_df.count() # action 메소드 이다
# class_df.show(1)
# class_df.show(2, truncate=2) # truncate : 각 열별 값을 몇글자 표현할 것인가 
# class_df.show(2, truncate=False) # 각 열의 값이 잘리는 것을 방지하겠다

class_df.show(2, vertical=True) # 각 행별로  
type(class_df.show(2, vertical=True)) # NoneType : collect() 같은 경우 출력후 list 반환인, show() 반환값없이 보여주기만 함

# 이 코드는 모든 node data 순회해서 결과 반환하므로 성능이 떨어지게 됨, action method + action method
# 사용할 일이 자주 있다면 메모리에 상주시켜 사용하는 것이 성능 측면에서 좋다
# class_df.show(class_df.count())


102

-RECORD 0-------------------
 class_cd      | 6OL        
 school        | ANKYI      
 class_std_cnt | 20         
 loc           | Urban      
 school_type   | Non-public 
 teaching_type | Standard   
-RECORD 1-------------------
 class_cd      | ZNS        
 school        | ANKYI      
 class_std_cnt | 21         
 loc           | Urban      
 school_type   | Non-public 
 teaching_type | Standard   
only showing top 2 rows

-RECORD 0-------------------
 class_cd      | 6OL        
 school        | ANKYI      
 class_std_cnt | 20         
 loc           | Urban      
 school_type   | Non-public 
 teaching_type | Standard   
-RECORD 1-------------------
 class_cd      | ZNS        
 school        | ANKYI      
 class_std_cnt | 21         
 loc           | Urban      
 school_type   | Non-public 
 teaching_type | Standard   
only showing top 2 rows



NoneType

## pyspark.sql.dataframe.DataFrame

- withColumn('컬럼명', '값')
    - 지연연산 모듈
    - 연산계획에 참여함
- 기존 컬럼 업데이트, 타입 변경, 신규컬럼 추가 기능
- 신규 또는 업데이트 값 줄때 주의
    - 신규컬럼 값 타입 활용, 업데이트 진행시, 기존 컬럼 값 활용 : col("컬럼명") -> 기존 컬럼의 값 참조 가능

- withColumnRenamed()
    - 컬럼명 변경 연산의 메소드

- spark의 lit function
    - 모든 타입을 허용하는 함수
    - 어떤 타입이 들어와도 객체로 인식하고 연산 진행시, 원하는 타입(유추타입)으로 변환 가능
    - 지연연산을 진행하므로 추가되는 컬럼값에 대해 타입 바로 결정 불가능, lit() 이용 객체 등록 후, 최적화 시, 타입 유추

In [23]:
data = [
    ('김철수', 15, date(2022,7,22), ('010','1111','2222')),
    ('이제동', 20, date(2021,7,22), ('010','2222','3333')),
    ('김명운', 25, date(2020,7,22), ('010','4444','5555')),
    ('홍진호', 36, date(2018,7,22), ('010','3333','4444'))
]
schema = StructType([
    StructField('name',StringType(),False,{'desc':'이름'}),
    StructField('age',IntegerType(),False,{'desc':'나이'}),    
    StructField('birth',DateType(),False,{'desc' :'생일'}),
    StructField('phone', StructType([
        StructField('phone1',StringType(),True),
        StructField('phone2',StringType(),True),
        StructField('phone3',StringType(),True)]),False,{'desc':'전화번호'}) # 중첩스키마
])
col_df = spark.createDataFrame(data, schema=schema)
type(col_df)

pyspark.sql.dataframe.DataFrame

In [24]:
col_df.show(3)

+------+---+----------+-----------------+
|  name|age|     birth|            phone|
+------+---+----------+-----------------+
|김철수| 15|2022-07-22|{010, 1111, 2222}|
|이제동| 20|2021-07-22|{010, 2222, 3333}|
|김명운| 25|2020-07-22|{010, 4444, 5555}|
+------+---+----------+-----------------+
only showing top 3 rows



In [29]:
# withColumn : 컬럼이름, 컬럼
# lit : column 객체를 literal로 만들어주는 함수
# 원하는 컬럼을 DataFrame에 추가
# col_df.withColumn('우승여부', '').show() # 이렇게 lit()으로 인지시켜주지 않으면 유추 자체가 불가능해짐
tmp = col_df.withColumn('우승여부', lit('')) # 그냥 객체가 있다 라고만 알려줘도 알아서 유추해줌
tmp.show()

+------+---+----------+-----------------+--------+
|  name|age|     birth|            phone|우승여부|
+------+---+----------+-----------------+--------+
|김철수| 15|2022-07-22|{010, 1111, 2222}|        |
|이제동| 20|2021-07-22|{010, 2222, 3333}|        |
|김명운| 25|2020-07-22|{010, 4444, 5555}|        |
|홍진호| 36|2018-07-22|{010, 3333, 4444}|        |
+------+---+----------+-----------------+--------+



In [31]:
# 값을 지정해서 추가
col_df.show()
col_df.withColumn("우승여부", lit("우승")).show() # 객체도 이렇게 lit()으로 인지시켜주지 않으면 유추 자체가 불가능해짐

+------+---+----------+-----------------+
|  name|age|     birth|            phone|
+------+---+----------+-----------------+
|김철수| 15|2022-07-22|{010, 1111, 2222}|
|이제동| 20|2021-07-22|{010, 2222, 3333}|
|김명운| 25|2020-07-22|{010, 4444, 5555}|
|홍진호| 36|2018-07-22|{010, 3333, 4444}|
+------+---+----------+-----------------+

+------+---+----------+-----------------+--------+
|  name|age|     birth|            phone|우승여부|
+------+---+----------+-----------------+--------+
|김철수| 15|2022-07-22|{010, 1111, 2222}|    우승|
|이제동| 20|2021-07-22|{010, 2222, 3333}|    우승|
|김명운| 25|2020-07-22|{010, 4444, 5555}|    우승|
|홍진호| 36|2018-07-22|{010, 3333, 4444}|    우승|
+------+---+----------+-----------------+--------+



### 파생컬럼 생성
- 기존 컬럼값을 가공해서 새로운 컬럼을 추가
- age 컬럼값에 따라 연령대라는 컬럼을 추가
- spark 의 sql.dataframe에서 
    - if문처럼 사용할 수 있는 SQL function인 case-when과 비슷한
        - when( + otherwise)을 사용 가능
        - when(조건1, 조건1이 참일때 값).when(조건2, 조건2가 참일때).otherwise(그 외 모든 경우 값)
        - 조건1이 참이면 마감, 거짓이면 --> 조건2가 참인지, 거짓이면 --> otherwise 까지
        - if - elif - else 

In [32]:
# when - otherwise : 조건에 따라 원하는 컬럼객체를 반환
col_df.age
col_df.printSchema()

Column<'age'>

root
 |-- name: string (nullable = false)
 |-- age: integer (nullable = false)
 |-- birth: date (nullable = false)
 |-- phone: struct (nullable = false)
 |    |-- phone1: string (nullable = true)
 |    |-- phone2: string (nullable = true)
 |    |-- phone3: string (nullable = true)



### column  내용  변경

In [44]:
# when - otherwise : 조건에 따라 원하는 컬럼객체를 반환
# 확인사항
# col_df.age => column<'age'> column 객체와 연산이 가능한지 O 
# 같은 것 col("age") == col_df.age == column("age")
# 정수 / 정수 = 정수의 결과값을 내놓는지  X // 15/10 이 실수인 1.5(정수가 X) 
# --> 버림으로 정수가 되도록 조치 결과값에 floor() 적용

tmp = col_df.withColumn("연령대", when(floor(col_df.age/10)==1, "10대") 
                                .when(floor(col_df.age/10)==2, "20대")
                                .otherwise("30대 이상"))
tmp.show()

+------+---+----------+-----------------+---------+
|  name|age|     birth|            phone|   연령대|
+------+---+----------+-----------------+---------+
|김철수| 15|2022-07-22|{010, 1111, 2222}|     10대|
|이제동| 20|2021-07-22|{010, 2222, 3333}|     20대|
|김명운| 25|2020-07-22|{010, 4444, 5555}|     20대|
|홍진호| 36|2018-07-22|{010, 3333, 4444}|30대 이상|
+------+---+----------+-----------------+---------+



In [50]:
# 기존 컬럼인 연령대의 값을 변경
tmp.show()
# tmp.withColumn("연령대", when(floor(col("age")/10)==1, "청소년")
#                        .when(floor(col("age")/10)==2, "청년")
#                        .otherwise("성인")).show()
tmp.withColumn("연령대", when(floor(col_df.age/10)==1, "청소년")
                       .when(floor(col_df.age/10)==2, "청년")
                       .otherwise("성인")).show()

+------+---+----------+-----------------+---------+
|  name|age|     birth|            phone|   연령대|
+------+---+----------+-----------------+---------+
|김철수| 15|2022-07-22|{010, 1111, 2222}|     10대|
|이제동| 20|2021-07-22|{010, 2222, 3333}|     20대|
|김명운| 25|2020-07-22|{010, 4444, 5555}|     20대|
|홍진호| 36|2018-07-22|{010, 3333, 4444}|30대 이상|
+------+---+----------+-----------------+---------+

+------+---+----------+-----------------+------+
|  name|age|     birth|            phone|연령대|
+------+---+----------+-----------------+------+
|김철수| 15|2022-07-22|{010, 1111, 2222}|청소년|
|이제동| 20|2021-07-22|{010, 2222, 3333}|  청년|
|김명운| 25|2020-07-22|{010, 4444, 5555}|  청년|
|홍진호| 36|2018-07-22|{010, 3333, 4444}|  성인|
+------+---+----------+-----------------+------+



### column 이름 변경
- withColumnRenamed("변경 전 컬럼명", "변경 후 컬럼명")

In [52]:
tmp.withColumnRenamed("연령대", "분류").show()

+------+---+----------+-----------------+---------+
|  name|age|     birth|            phone|     분류|
+------+---+----------+-----------------+---------+
|김철수| 15|2022-07-22|{010, 1111, 2222}|     10대|
|이제동| 20|2021-07-22|{010, 2222, 3333}|     20대|
|김명운| 25|2020-07-22|{010, 4444, 5555}|     20대|
|홍진호| 36|2018-07-22|{010, 3333, 4444}|30대 이상|
+------+---+----------+-----------------+---------+



### column  삭제 연산 계획
- spark.sql.df.drop("컬럼명")
- 해당 컬럼이 없어도 연산 처리 시(action), 에러가 발생하지 않고 넘어감

In [54]:
tmp = tmp.drop("분류") 
tmp.show()

+------+---+----------+-----------------+---------+
|  name|age|     birth|            phone|   연령대|
+------+---+----------+-----------------+---------+
|김철수| 15|2022-07-22|{010, 1111, 2222}|     10대|
|이제동| 20|2021-07-22|{010, 2222, 3333}|     20대|
|김명운| 25|2020-07-22|{010, 4444, 5555}|     20대|
|홍진호| 36|2018-07-22|{010, 3333, 4444}|30대 이상|
+------+---+----------+-----------------+---------+



In [55]:
tmp = tmp.withColumnRenamed("연령대", "분류")
tmp = tmp.drop("분류") 
tmp.show()

+------+---+----------+-----------------+
|  name|age|     birth|            phone|
+------+---+----------+-----------------+
|김철수| 15|2022-07-22|{010, 1111, 2222}|
|이제동| 20|2021-07-22|{010, 2222, 3333}|
|김명운| 25|2020-07-22|{010, 4444, 5555}|
|홍진호| 36|2018-07-22|{010, 3333, 4444}|
+------+---+----------+-----------------+



# 2. DataFrame 사용 하기

참고 : https://spark.apache.org/docs/3.2.0/api/scala/org/apache/spark/sql/Dataset.html 

- DataFrame의 메서드의 구분
 - transformation
 - action
 - Basic Dataset functions  
 
 
- DataFrame의 사용은 SQL 쿼리 구조를 따라간다